# GAN-Based Image Generation

This notebook implements a Generative Adversarial Network (GAN) for generating synthetic images based on training data.

## What You'll Learn:
1. Build a Generator network to create images from noise
2. Build a Discriminator network to distinguish real vs fake images
3. Train both networks adversarially
4. Generate new images based on learned patterns
5. Visualize training progress and results

---

## 1. Setup and Imports

In [2]:
# Import required libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# TensorFlow and Keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.preprocessing.image import load_img, img_to_array, array_to_img

# Image processing
from PIL import Image
import cv2

# Progress bar

from tqdm import tqdm

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"Keras version: {keras.__version__}")

TensorFlow version: 2.20.0
GPU Available: False
Keras version: 3.12.0


## 2. Configuration

In [ ]:
# Training configuration
IMG_HEIGHT = 64
IMG_WIDTH = 64
IMG_CHANNELS = 3
LATENT_DIM = 100  # Dimension of noise vector

BATCH_SIZE = 32
EPOCHS = 100
LEARNING_RATE = 0.0002
BETA_1 = 0.5  # Adam optimizer parameter

# Directories
PROJECT_ROOT = Path().absolute().parent
DATA_DIR = PROJECT_ROOT / "data"
MODELS_DIR = PROJECT_ROOT / "models"
RESULTS_DIR = PROJECT_ROOT / "results"

# Create directories if they don't exist
DATA_DIR.mkdir(exist_ok=True)
MODELS_DIR.mkdir(exist_ok=True)
RESULTS_DIR.mkdir(exist_ok=True)

print("Configuration:")
print(f"  Image size: {IMG_HEIGHT}x{IMG_WIDTH}x{IMG_CHANNELS}")
print(f"  Latent dimension: {LATENT_DIM}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Epochs: {EPOCHS}")
print(f"  Learning rate: {LEARNING_RATE}")

## 3. Data Loading and Preprocessing

Load your training images from the data directory. You can use any dataset of images.

In [ ]:
def load_and_preprocess_images(data_path, img_height=64, img_width=64):
    """
    Load images from directory and preprocess them.
    
    Args:
        data_path: Path to directory containing images
        img_height: Target image height
        img_width: Target image width
    
    Returns:
        Preprocessed images as numpy array
    """
    images = []
    valid_extensions = {'.jpg', '.jpeg', '.png', '.bmp'}
    
    image_files = [f for f in os.listdir(data_path) 
                   if Path(f).suffix.lower() in valid_extensions]
    
    if not image_files:
        print("⚠️  No images found in data directory!")
        print("   Using synthetic data for demonstration...")
        # Generate synthetic data for demo
        return generate_synthetic_data(1000, img_height, img_width)
    
    print(f"Loading {len(image_files)} images...")
    
    for img_file in tqdm(image_files):
        try:
            img_path = os.path.join(data_path, img_file)
            img = load_img(img_path, target_size=(img_height, img_width))
            img_array = img_to_array(img)
            images.append(img_array)
        except Exception as e:
            print(f"Error loading {img_file}: {e}")
    
    if not images:
        return generate_synthetic_data(1000, img_height, img_width)
    
    # Convert to numpy array and normalize to [-1, 1]
    images = np.array(images)
    images = (images - 127.5) / 127.5
    
    return images

def generate_synthetic_data(num_samples, height, width):
    """
    Generate synthetic training data for demonstration.
    Creates simple geometric patterns.
    """
    print(f"Generating {num_samples} synthetic images...")
    images = []
    
    for i in range(num_samples):
        # Create image with random colors and simple shapes
        img = np.random.rand(height, width, 3) * 0.3
        
        # Add random circles
        num_circles = np.random.randint(2, 6)
        for _ in range(num_circles):
            cx, cy = np.random.randint(0, width), np.random.randint(0, height)
            radius = np.random.randint(5, 20)
            color = np.random.rand(3)
            
            y, x = np.ogrid[:height, :width]
            mask = (x - cx)**2 + (y - cy)**2 <= radius**2
            img[mask] = color
        
        images.append(img)
    
    images = np.array(images)
    # Normalize to [-1, 1]
    images = (images - 0.5) / 0.5
    
    return images

# Load training data
train_images = load_and_preprocess_images(DATA_DIR, IMG_HEIGHT, IMG_WIDTH)

print(f"\nDataset shape: {train_images.shape}")
print(f"Value range: [{train_images.min():.2f}, {train_images.max():.2f}]")
print(f"Number of training images: {len(train_images)}")

### Visualize Sample Images

In [ ]:
def plot_images(images, n_rows=2, n_cols=5, title="Sample Images"):
    """
    Plot a grid of images.
    """
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6))
    fig.suptitle(title, fontsize=16, fontweight='bold')
    
    for i, ax in enumerate(axes.flat):
        if i < len(images):
            # Denormalize from [-1, 1] to [0, 1]
            img = (images[i] + 1) / 2.0
            img = np.clip(img, 0, 1)
            ax.imshow(img)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# Display sample training images
sample_indices = np.random.choice(len(train_images), 10, replace=False)
plot_images(train_images[sample_indices], title="Training Data Samples")

## 4. Build the Generator

The Generator takes random noise and transforms it into an image.

In [ ]:
def build_generator(latent_dim, img_shape):
    """
    Build the Generator model.
    
    Args:
        latent_dim: Dimension of input noise vector
        img_shape: Output image shape (height, width, channels)
    
    Returns:
        Generator model
    """
    model = models.Sequential(name='Generator')
    
    # Foundation: Start with dense layer
    n_nodes = 8 * 8 * 256
    model.add(layers.Dense(n_nodes, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Reshape((8, 8, 256)))
    
    # Upsample to 16x16
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    
    # Upsample to 32x32
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    
    # Upsample to 64x64
    model.add(layers.Conv2DTranspose(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha=0.2))
    
    # Output layer
    model.add(layers.Conv2D(img_shape[2], (7, 7), activation='tanh', padding='same'))
    
    return model

# Build and display generator
generator = build_generator(LATENT_DIM, (IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
generator.summary()

# Test generator output
noise = tf.random.normal([1, LATENT_DIM])
generated_image = generator(noise, training=False)
print(f"\nGenerated image shape: {generated_image.shape}")

## 5. Build the Discriminator

The Discriminator classifies images as real or fake.

In [ ]:
def build_discriminator(img_shape):
    """
    Build the Discriminator model.
    
    Args:
        img_shape: Input image shape (height, width, channels)
    
    Returns:
        Discriminator model
    """
    model = models.Sequential(name='Discriminator')
    
    # Input layer
    model.add(layers.Input(shape=img_shape))
    
    # Downsample 64x64 -> 32x32
    model.add(layers.Conv2D(64, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    
    # Downsample 32x32 -> 16x16
    model.add(layers.Conv2D(128, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    
    # Downsample 16x16 -> 8x8
    model.add(layers.Conv2D(256, (4, 4), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.3))
    
    # Classifier
    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation='sigmoid'))
    
    return model

# Build and display discriminator
discriminator = build_discriminator((IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS))
discriminator.summary()

# Test discriminator output
decision = discriminator(generated_image)
print(f"\nDiscriminator decision shape: {decision.shape}")
print(f"Decision value (0=fake, 1=real): {decision.numpy()[0][0]:.4f}")

## 6. Define Loss Functions and Optimizers

In [ ]:
# Binary cross-entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    """
    Discriminator loss: 
    - Real images should be classified as 1
    - Fake images should be classified as 0
    """
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    """
    Generator loss:
    - Fake images should fool the discriminator (classified as 1)
    """
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Optimizers
generator_optimizer = optimizers.Adam(LEARNING_RATE, beta_1=BETA_1)
discriminator_optimizer = optimizers.Adam(LEARNING_RATE, beta_1=BETA_1)

print("Loss functions and optimizers configured.")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Beta_1: {BETA_1}")

## 7. Training Step

In [ ]:
@tf.function
def train_step(images):
    """
    Single training step for both generator and discriminator.
    """
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images
        generated_images = generator(noise, training=True)
        
        # Discriminator predictions
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        # Calculate losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)
    
    # Calculate gradients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    
    # Apply gradients
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    
    return gen_loss, disc_loss

print("Training step function defined.")

## 8. Training Loop

In [ ]:
def generate_and_save_images(model, epoch, test_input, save_dir):
    """
    Generate images and save them to disk.
    """
    predictions = model(test_input, training=False)
    
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle(f'Generated Images - Epoch {epoch}', fontsize=16, fontweight='bold')
    
    for i, ax in enumerate(axes.flat):
        if i < len(predictions):
            # Denormalize
            img = (predictions[i] + 1) / 2.0
            img = np.clip(img, 0, 1)
            ax.imshow(img)
        ax.axis('off')
    
    plt.tight_layout()
    save_path = save_dir / f'image_at_epoch_{epoch:04d}.png'
    plt.savefig(save_path, dpi=100, bbox_inches='tight')
    plt.show()
    print(f"Saved: {save_path}")

def train_gan(dataset, epochs):
    """
    Train the GAN for specified number of epochs.
    """
    # Create dataset
    train_dataset = tf.data.Dataset.from_tensor_slices(dataset).shuffle(1000).batch(BATCH_SIZE)
    
    # Fixed seed for consistent visualization
    seed = tf.random.normal([10, LATENT_DIM])
    
    # Track losses
    gen_losses = []
    disc_losses = []
    
    print("\n" + "="*80)
    print("STARTING GAN TRAINING")
    print("="*80)
    print(f"Total epochs: {epochs}")
    print(f"Batch size: {BATCH_SIZE}")
    print(f"Steps per epoch: {len(train_dataset)}")
    print("="*80 + "\n")
    
    for epoch in range(epochs):
        epoch_gen_loss = []
        epoch_disc_loss = []
        
        # Training progress bar
        pbar = tqdm(train_dataset, desc=f'Epoch {epoch+1}/{epochs}')
        
        for image_batch in pbar:
            gen_loss, disc_loss = train_step(image_batch)
            epoch_gen_loss.append(gen_loss.numpy())
            epoch_disc_loss.append(disc_loss.numpy())
            
            # Update progress bar
            pbar.set_postfix({
                'G_loss': f'{gen_loss.numpy():.4f}',
                'D_loss': f'{disc_loss.numpy():.4f}'
            })
        
        # Average losses for epoch
        avg_gen_loss = np.mean(epoch_gen_loss)
        avg_disc_loss = np.mean(epoch_disc_loss)
        gen_losses.append(avg_gen_loss)
        disc_losses.append(avg_disc_loss)
        
        # Print epoch summary
        print(f"\nEpoch {epoch+1} Summary:")
        print(f"  Generator Loss: {avg_gen_loss:.4f}")
        print(f"  Discriminator Loss: {avg_disc_loss:.4f}")
        
        # Generate and save images every 10 epochs
        if (epoch + 1) % 10 == 0 or epoch == 0:
            print(f"\nGenerating sample images...")
            generate_and_save_images(generator, epoch + 1, seed, RESULTS_DIR)
        
        # Save model checkpoint every 20 epochs
        if (epoch + 1) % 20 == 0:
            checkpoint_path = MODELS_DIR / f'generator_epoch_{epoch+1}.h5'
            generator.save(checkpoint_path)
            print(f"\n💾 Model saved: {checkpoint_path}")
        
        print("-" * 80)
    
    print("\n" + "="*80)
    print("✅ TRAINING COMPLETED!")
    print("="*80)
    
    return gen_losses, disc_losses

# Start training
gen_losses, disc_losses = train_gan(train_images, EPOCHS)

## 9. Visualize Training Progress

In [ ]:
# Plot training losses
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(gen_losses, label='Generator Loss', color='blue', linewidth=2)
plt.plot(disc_losses, label='Discriminator Loss', color='red', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Training Losses Over Time', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
plt.plot(gen_losses, label='Generator Loss', color='blue', linewidth=2)
plt.xlabel('Epoch', fontsize=12)
plt.ylabel('Loss', fontsize=12)
plt.title('Generator Loss', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)

plt.tight_layout()
loss_plot_path = RESULTS_DIR / 'training_losses.png'
plt.savefig(loss_plot_path, dpi=150, bbox_inches='tight')
plt.show()

print(f"\n📊 Loss plot saved: {loss_plot_path}")
print(f"\nFinal Generator Loss: {gen_losses[-1]:.4f}")
print(f"Final Discriminator Loss: {disc_losses[-1]:.4f}")

## 10. Generate New Images

In [ ]:
def generate_images(generator, num_images=10, save=True):
    """
    Generate new images using the trained generator.
    """
    # Generate random noise
    noise = tf.random.normal([num_images, LATENT_DIM])
    
    # Generate images
    generated_images = generator(noise, training=False)
    
    # Plot results
    n_rows = 2
    n_cols = 5
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6))
    fig.suptitle('Generated Images (Final Model)', fontsize=16, fontweight='bold')
    
    for i, ax in enumerate(axes.flat):
        if i < len(generated_images):
            # Denormalize
            img = (generated_images[i] + 1) / 2.0
            img = np.clip(img, 0, 1)
            ax.imshow(img)
            
            if save:
                # Save individual image
                img_pil = array_to_img(img)
                img_path = RESULTS_DIR / f'generated_image_{i+1}.png'
                img_pil.save(img_path)
        ax.axis('off')
    
    plt.tight_layout()
    grid_path = RESULTS_DIR / 'final_generated_images.png'
    plt.savefig(grid_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    if save:
        print(f"\n✅ Generated {num_images} images")
        print(f"📁 Saved to: {RESULTS_DIR}")
    
    return generated_images

# Generate final images
print("\nGenerating final images...")
final_images = generate_images(generator, num_images=10, save=True)

## 11. Save Final Model

In [ ]:
# Save the trained models
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

generator_path = MODELS_DIR / f'generator_final_{timestamp}.h5'
discriminator_path = MODELS_DIR / f'discriminator_final_{timestamp}.h5'

generator.save(generator_path)
discriminator.save(discriminator_path)

print("\n" + "="*80)
print("✅ MODELS SAVED SUCCESSFULLY")
print("="*80)
print(f"Generator: {generator_path}")
print(f"Discriminator: {discriminator_path}")
print("="*80)

## 12. Load and Use Saved Model

In [ ]:
# Example: Load a saved generator and generate images
# Uncomment and update the path to use

# loaded_generator = keras.models.load_model(generator_path)
# print("Generator loaded successfully!")

# # Generate new images with loaded model
# generate_images(loaded_generator, num_images=5, save=False)

## 13. Summary and Next Steps

### What We Accomplished:
1. ✅ Built a complete GAN architecture (Generator + Discriminator)
2. ✅ Trained the model on image data
3. ✅ Generated synthetic images from random noise
4. ✅ Visualized training progress
5. ✅ Saved trained models for future use

### To Improve Results:
- **More Training Data**: Use larger, more diverse image datasets
- **Longer Training**: Increase epochs (200-500+)
- **Architecture Tweaks**: Experiment with layer sizes and depths
- **Advanced Techniques**: Try Progressive GAN, StyleGAN, or Conditional GAN
- **Hyperparameter Tuning**: Adjust learning rates, batch sizes, etc.

### Common Issues:
- **Mode Collapse**: Generator produces limited variety → Reduce learning rate
- **Training Instability**: Losses oscillate wildly → Add gradient penalty, use label smoothing
- **Poor Quality**: Blurry or unrealistic images → Train longer, use more data

### Next Steps:
1. Experiment with your own image datasets
2. Try conditional GANs for controlled generation
3. Explore image-to-image translation tasks
4. Implement advanced GAN architectures (DCGAN, WGAN, StyleGAN)

---

**Happy experimenting with GANs! 🎨🤖**